In [21]:
# phase1_apify_client.py

import os
import pandas as pd
from dotenv import load_dotenv
from apify_client import ApifyClient

# Load environment variables
load_dotenv()
API_TOKEN = os.getenv("APIFY_API_TOKEN")

if not API_TOKEN:
    raise RuntimeError("❌ APIFY_API_TOKEN not set in .env file.")

# Initialize Apify client
client = ApifyClient(API_TOKEN)

def run_actor_search(search_url: str, max_items: int = 100):
    """Triggers the Apify actor with the Facebook Ads Library search URL."""
    print("▶️ Starting Apify actor run...")
    run_input = {
        "urls": [{"url": search_url}],
        "count": max_items,
        "scrapePageAds.activeStatus": "active",
    }
    run = client.actor("curious_coder/facebook-ads-library-scraper").call(run_input=run_input)
    dataset_id = run.get("defaultDatasetId")
    if not dataset_id:
        raise Exception("❌ Dataset ID not returned from actor.")
    return dataset_id

def fetch_dataset(dataset_id):
    """Fetches all results from the Apify dataset."""
    print(f"📥 Fetching results from dataset: {dataset_id}")
    return list(client.dataset(dataset_id).iterate_items())

def clean_ads(data):
    """Deduplicates and cleans ad data for export."""
    seen = set()
    clean = []
    for ad in data:
        page = ad.get("pageName") or ad.get("page_name")
        url = ad.get("pageUrl")
        if not url and page:
            url = f"https://www.facebook.com/{page.replace(' ', '')}"

        # Extract ad text safely
        ad_text = ad.get("adText", "")
        if not ad_text:
            snapshot = ad.get("snapshot") or {}
            body = snapshot.get("body") or {}
            ad_text = body.get("text", "")

        if page and page not in seen:
            clean.append({
                "page_name": page,
                "page_url": url,
                "ad_text": ad_text.strip() or "N/A"
            })
            seen.add(page)
    return pd.DataFrame(clean)

def main():
    print("🟢 Meta Ads Scraper: Phase 1 (Extraction)")

    # Get user input
    keyword = input("🔤 Enter keyword (e.g., 'marketing'): ").strip()
    country = input("🌍 Enter 2-letter country code (e.g., 'US'): ").strip().upper()

    if not keyword or not country:
        print("❗ Both keyword and country are required.")
        return

    # Build Facebook Ads Library URL
    fb_url = (
        "https://www.facebook.com/ads/library/"
        f"?active_status=active&ad_type=all&country={country}"
        f"&q={keyword}&search_type=keyword_unordered&media_type=all"
    )

    print(f"🔗 Facebook Ads Library URL: {fb_url}")

    try:
        dataset_id = run_actor_search(fb_url)
        raw_ads = fetch_dataset(dataset_id)
        print(f"📊 Retrieved {len(raw_ads)} total ads")

        df = clean_ads(raw_ads)
        print(f"🧹 Cleaned to {len(df)} unique pages")

        os.makedirs("results", exist_ok=True)
        output_path = "results/ads_cleaned.csv"
        df.to_csv(output_path, index=False, encoding="utf-8")
        print(f"✅ Phase 1 complete — saved to: {output_path}")

    except Exception as e:
        print("❌ Error occurred:", e)


In [22]:

if __name__ == "__main__":
    main()


🟢 Meta Ads Scraper: Phase 1 (Extraction)
🔗 Facebook Ads Library URL: https://www.facebook.com/ads/library/?active_status=active&ad_type=all&country=US&q=marketing&search_type=keyword_unordered&media_type=all
▶️ Starting Apify actor run...


[apify.facebook-ads-library-scraper runId:LhD1aV8P8KaTYJHFj] -> Status: RUNNING, Message: 
[apify.facebook-ads-library-scraper runId:LhD1aV8P8KaTYJHFj] -> 2025-06-19T12:18:38.085Z ACTOR: Pulling Docker image of build kUzwzTlTV9cnZ9bg8 from registry.
[apify.facebook-ads-library-scraper runId:LhD1aV8P8KaTYJHFj] -> 2025-06-19T12:18:38.087Z ACTOR: Creating Docker container.
[apify.facebook-ads-library-scraper runId:LhD1aV8P8KaTYJHFj] -> 2025-06-19T12:18:38.132Z ACTOR: Starting Docker container.
[apify.facebook-ads-library-scraper runId:LhD1aV8P8KaTYJHFj] -> 2025-06-19T12:18:41.258Z INFO  System info {"apifyVersion":"3.2.6","apifyClientVersion":"2.10.0","crawleeVersion":"3.12.0","osType":"Linux","nodeVersion":"v20.19.0"}
[apify.facebook-ads-library-scraper runId:LhD1aV8P8KaTYJHFj] -> 2025-06-19T12:18:56.719Z ℹ️ Crawler statistics:
[apify.facebook-ads-library-scraper runId:LhD1aV8P8KaTYJHFj] -> 2025-06-19T12:18:56.721Z 	Tried requests: 1
[apify.facebook-ads-library-scraper runId:LhD1aV8P8KaT

📥 Fetching results from dataset: ysl7D5q82s1Eyg0fF
📊 Retrieved 100 total ads
🧹 Cleaned to 56 unique pages
✅ Phase 1 complete — saved to: results/ads_cleaned.csv
